In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import os, sys

In [3]:
import pylab as plt
import matplotlib

import numpy as np
import pandas as pd

In [4]:
sys.path.append('..')

from pyMultiOmics.base import SingleOmicsData, MultiOmicsData
from pyMultiOmics.constants import HOMO_SAPIENS, PROTEINS, COMPOUNDS, REACTIONS
from pyMultiOmics.mapping import Mapper
from pyMultiOmics.common import set_log_level_info, set_log_level_debug, download_file, extract_zip_file

2022-03-24 14:40:01.676 | WARNING  | pyMultiOmics.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2022-03-24 14:40:01.676 | WARNING  | pyMultiOmics.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2022-03-24 14:40:01.677 | INFO     | pyMultiOmics.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# Demonstration of pyMultiOmics mapping

## Load the processed Covid data from [1]

[1] [Shen, Bo, et al. "Proteomic and metabolomic characterization of COVID-19 patient sera." Cell 182.1 (2020): 59-72.](https://www.sciencedirect.com/science/article/pii/S0092867420306279?casa_token=wKwWn9P4MK0AAAAA:v8z5MVnQ1ONrcatncCsigSDoxeOq2ZOcN4da9SofGDcpeDqrC76QK8yNKrKtVUrMWBBAntI8)

In [5]:
url = 'https://github.com/glasgowcompbio/pyMultiOmics-data/raw/main/covid19_dualomics_data.zip'
out_file = download_file(url)
extract_zip_file(out_file)

2022-03-24 14:40:02.949 | INFO     | pyMultiOmics.common:download_file:59 - Downloading covid19_dualomics_data.zip
551KB [00:00, 8.89kKB/s]                                                                                                
2022-03-24 14:40:03.061 | INFO     | pyMultiOmics.common:extract_zip_file:71 - Extracting covid19_dualomics_data.zip
100%|█████████████████████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 1191.03it/s]
2022-03-24 14:40:03.075 | INFO     | pyMultiOmics.common:extract_zip_file:77 - Deleting covid19_dualomics_data.zip


In [6]:
DATA_FOLDER = os.path.abspath(os.path.join('covid19_dualomics_data'))
DATA_FOLDER

'/Users/joewandy/Work/git/pyMultiOmics/notebooks/covid19_dualomics_data'

Read proteomics data

In [7]:
protein_df = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_data.csv'), index_col='Identifier')
protein_design = pd.read_csv(os.path.join(DATA_FOLDER, 'protein_design.csv'), index_col='sample')

In [8]:
protein_df.head()

,h_F1_131N,h_F1_131C,h_F1_132C,h_F2_131N,h_F2_131C,h_F2_132C,h_F3_131N,h_F3_131C,h_F3_132C,h_F4_131N,...,s_F3_128N,s_F3_128C,s_F3_129C,s_F4_128N,s_F4_128C,s_F5_128N,s_F5_128C,s_F6_128N,s_F6_128C,s_F6_133N
Identifier,,,,,,,,,,,,,,,,,,,,,
P04114,0.750,0.853,0.822,1.191,1.175,1.078,0.693,0.947,0.931,1.057,...,1.044,1.305,1.657,1.323,1.624,1.170,0.981,0.791,1.029,1.195
P01024,0.782,1.057,0.994,0.864,0.917,0.790,0.823,1.152,0.816,0.920,...,1.100,0.986,1.114,1.210,1.289,1.104,1.111,1.007,1.159,0.979
P02768,1.183,1.101,1.045,1.086,1.041,1.187,1.234,1.079,1.011,1.099,...,0.786,0.706,0.947,0.831,0.717,0.795,0.776,0.938,0.903,0.743
P01023,1.066,1.278,0.959,0.811,0.789,0.931,0.971,0.769,1.011,0.866,...,0.817,0.728,0.861,0.798,0.751,0.917,0.809,0.780,1.195,0.706
P02751,1.085,0.947,0.993,1.343,1.130,0.778,0.731,1.084,1.107,0.909,...,0.566,0.854,1.109,0.630,0.850,0.661,0.848,0.829,0.760,0.811


In [9]:
protein_design

,group
sample,
h_F1_131N,healthy
h_F1_131C,healthy
h_F1_132C,healthy
h_F2_131N,healthy
h_F2_131C,healthy
...,...
s_F5_128N,severe
s_F5_128C,severe
s_F6_128N,severe


Read metabolomics data

In [10]:
compound_df = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_data.csv'), index_col='Identifier')
compound_design = pd.read_csv(os.path.join(DATA_FOLDER, 'compound_design.csv'), index_col='sample')

In [11]:
compound_df.head()

,h_jkdz1,h_jkdz2,h_jkdz3,h_jkdz4,h_jkdz5,h_jkdz6,h_jkdz7,h_jkdz8,h_jkdz9,h_jkdz10,...,s_ZX12,s_ZX13,s_ZX14,s_ZX15,s_ZX16,s_ZX17,s_ZX18,s_ZX19,s_ZX20,s_ZX21
Identifier,,,,,,,,,,,,,,,,,,,,,
C21482,19413052.00,6.381812e+06,9.748316e+06,5.326872e+06,1.998072e+07,3.580375e+06,8.256121e+06,8.079382e+06,1.559659e+07,1.520363e+07,...,1.904349e+06,3.226016e+06,7.378147e+05,2.817698e+06,3.329101e+06,3206752.75,1.466174e+06,2.779301e+06,2.117668e+06,2.184310e+06
C18218,2711915.25,2.056393e+06,1.445594e+06,2.038765e+06,2.536996e+06,2.638198e+06,2.285757e+06,1.973140e+06,2.015425e+06,2.290842e+06,...,1.409720e+06,1.413307e+06,3.218834e+06,1.602131e+06,1.317878e+06,2930312.75,1.168094e+06,2.946776e+06,1.417311e+06,1.474166e+06
C05127,87727.25,NaN,9.238706e+04,NaN,1.597879e+05,NaN,NaN,NaN,9.055130e+04,1.214114e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.382788e+05,NaN,NaN
C01152,58832828.00,5.843934e+07,5.552133e+07,4.516214e+07,5.478952e+07,3.941259e+07,2.987876e+07,6.751726e+07,4.666031e+07,9.118524e+07,...,2.881314e+07,3.164358e+07,2.538767e+07,3.307604e+07,3.915698e+07,24400592.00,2.593375e+07,6.413868e+07,4.020588e+07,4.904488e+07
C02918,NaN,1.815549e+05,2.240392e+05,1.609397e+05,3.206194e+05,7.176557e+05,3.268182e+05,5.135810e+05,2.734582e+05,NaN,...,3.337245e+05,NaN,4.347152e+05,3.532118e+04,NaN,655827.25,8.359706e+05,4.034381e+06,2.839358e+05,8.062160e+04


In [12]:
compound_design

,group
sample,
h_jkdz1,healthy
h_jkdz2,healthy
h_jkdz3,healthy
h_jkdz4,healthy
h_jkdz5,healthy
...,...
s_ZX17,severe
s_ZX18,severe
s_ZX19,severe


In [13]:
set_log_level_info()

1

## Create multi-omics container object

In [14]:
protein_data = SingleOmicsData(PROTEINS, protein_df, protein_design)
protein_data

proteins data with (791, 70) measurements

In [15]:
compound_data = SingleOmicsData(COMPOUNDS, compound_df, compound_design)
compound_data

compounds data with (220, 96) measurements

In [16]:
publication = 'Proteomic and Metabolomic Characterization of COVID-19 Patient Sera'
url = 'https://www.cell.com/cell/fulltext/S0092-8674(20)30627-9'

In [17]:
mo = MultiOmicsData(publication=publication, url=url)
mo.add_data([protein_data, compound_data])
mo

Multi-omics data container
- publication: Proteomic and Metabolomic Characterization of COVID-19 Patient Sera
- URL: https://www.cell.com/cell/fulltext/S0092-8674(20)30627-9
- Views: 2 modalities
	 - proteins data with (791, 70) measurements
	 - compounds data with (220, 96) measurements

## Create a mapping object

The mapping object uses Reactome to map the different biological entities in the data:
- Transcripts (or genes) are connected to the proteins they encode
- Proteins and compounds are connected to reactions they're involved in
- Reactions are connected to pathways

In [18]:
m = Mapper(mo, HOMO_SAPIENS, metabolic_pathway_only=False)
m.build()

2022-03-24 14:40:03.534 | INFO     | pyMultiOmics.functions:remove_dupes:385 - Removing 9 rows with duplicate identifiers
2022-03-24 14:40:03.535 | INFO     | pyMultiOmics.functions:reactome_mapping:78 - There are 211 observed compound ids
2022-03-24 14:40:03.535 | INFO     | pyMultiOmics.functions:reactome_mapping:81 - Mapping genes -> proteins
2022-03-24 14:40:06.215 | INFO     | pyMultiOmics.functions:reactome_mapping:86 - Mapping proteins -> reactions
2022-03-24 14:40:07.489 | INFO     | pyMultiOmics.functions:reactome_mapping:94 - Mapping compounds -> reactions
2022-03-24 14:40:11.539 | INFO     | pyMultiOmics.functions:reactome_mapping:100 - Mapping reactions -> pathways
2022-03-24 14:40:12.580 | INFO     | pyMultiOmics.functions:reactome_mapping:111 - Mapping reactions -> proteins
2022-03-24 14:40:18.631 | INFO     | pyMultiOmics.functions:reactome_mapping:118 - Mapping reactions -> compounds
2022-03-24 14:40:21.469 | INFO     | pyMultiOmics.functions:reactome_mapping:130 - Mapp

In [19]:
m

## Query mapping object

Below shows some example queries we can perform with the mapping object

In [20]:
reactions = m.get_nodes(types=REACTIONS)

data = []
for reaction_id, reaction_data in reactions:
    reaction_name = reaction_data['display_name']
    proteins = m.get_connected(reaction_id, dest_type=PROTEINS, observed=True)
    compounds = m.get_connected(reaction_id, dest_type=COMPOUNDS, observed=True)
    
    if len(proteins) > 0 and len(compounds) > 0:
        row = [reaction_id, reaction_name, len(proteins), len(compounds)]
        data.append(row)

df = pd.DataFrame(data, columns=['reaction_id', 'reaction_name', 'num_proteins', 'num_compounds'])
df

,reaction_id,reaction_name,num_proteins,num_compounds
0,R-HSA-114552,Thrombin-activated pars activate g12/13,1,1
1,R-HSA-114558,Thrombin-activated pars activate gq,1,1
2,R-HSA-1214188,Prdm9 trimethylates histone h3,3,1
3,R-HSA-1605591,Glucosylceramidase cleaves the glucosidic bond...,1,1
4,R-HSA-163432,Cholesterol ester + h2o -> cholesterol + fatty...,1,1
...,...,...,...,...
223,R-HSA-9710490,The gsdme gene promoter is hypermethylated,3,1
224,R-HSA-9733545,Bile salts and acids bind alb,1,5
225,R-HSA-9733960,Bile salts and acids dissociate from alb,1,5
226,R-HSA-977071,Sialyltransferase i can add sialic acid to the...,1,1


##### List all entities connected to reaction R-HSA-194153

In [21]:
query_id = 'R-HSA-194153'
m.get_connected(query_id)

,display_name,data_type,observed,source_id
entity_id,,,,
ENSG00000108846,Abcc3,genes,False,R-HSA-194153
O15438,O15438,proteins,False,R-HSA-194153
28865,Taurocholic acid,compounds,True,R-HSA-194153
30616,Atp(4-),compounds,False,R-HSA-194153
17687,Glycocholic acid,compounds,False,R-HSA-194153
36274,Glycochenodeoxycholic acid,compounds,True,R-HSA-194153
16525,Taurochenodeoxycholic acid,compounds,False,R-HSA-194153
43474,Hydrogenphosphate,compounds,False,R-HSA-194153
456216,Adp(3-),compounds,False,R-HSA-194153


##### Query the connections between proteins and compounds (through their shared reactions)

In [22]:
query_id = 'P02768'
m.get_connected(query_id, dest_type=COMPOUNDS)

,display_name,data_type,observed,source_id
entity_id,,,,
18421,Superoxide,compounds,False,P02768
17033,Biliverdin,compounds,True,P02768
16990,Bilirubin ixalpha,compounds,True,P02768
17627,Ferroheme b,compounds,False,P02768
36144,Ferriheme b,compounds,False,P02768
456216,Adp(3-),compounds,False,P02768
30616,Atp(4-),compounds,False,P02768
36274,Glycochenodeoxycholic acid,compounds,True,P02768
16755,Chenodeoxycholic acid,compounds,True,P02768


In [23]:
query_id = '16990'
m.get_connected(query_id, dest_type=PROTEINS)

,display_name,data_type,observed,source_id
entity_id,,,,
Q9NPD5,Q9NPD5,proteins,False,16990
P33527,P33527,proteins,False,16990
Q9BYK8,Q9BYK8,proteins,False,16990
Q96RS0,Q96RS0,proteins,False,16990
Q92793,Q92793,proteins,False,16990
...,...,...,...,...
P08047,P08047,proteins,False,16990
P30043,P30043,proteins,True,16990
P53004,P53004,proteins,False,16990
